In [ ]:
%matplotlib inline
from func_adl_servicex import ServiceXSourceUpROOT

# compute dilepton mass (zero mass leptons)
def m(d, l1, l2):
    import numpy
    return numpy.sqrt(2*d[f'lep_pt_{l1}']*d[f'lep_pt_{l2}']*
                    (numpy.cosh(d[f'lep_eta_{l1}']-d[f'lep_eta_{l2}'])
                     -numpy.cos(d[f'lep_phi_{l1}']-d[f'lep_phi_{l2}'])))

In [ ]:
# Inclusive selection for lepton quantities and a few other things from a specific rucio dataset
# returns awkward array
dataset_uproot = "user.kchoi:user.kchoi.ttHML_80fb_ttZ"
ds = ServiceXSourceUpROOT(dataset_uproot, "nominal")
data = ds.Select("lambda e: {'lep_pt_0': e.lep_Pt_0/1000., \
                             'lep_pt_1': e.lep_Pt_1/1000., \
                             'lep_pt_2': e.lep_Pt_2/1000., \
                             'lep_eta_0': e.lep_Eta_0, \
                             'lep_eta_1': e.lep_Eta_1, \
                             'lep_eta_2': e.lep_Eta_2, \
                             'lep_phi_0': e.lep_Phi_0, \
                             'lep_phi_1': e.lep_Phi_1, \
                             'lep_phi_2': e.lep_Phi_2, \
                             'lep_id_0': e.lep_ID_0, \
                             'lep_id_1': e.lep_ID_1, \
                             'lep_id_2': e.lep_ID_2, \
                             'is3L': e.is3L, \
                             'mcWeight': e.mcWeightOrg, \
                             'nJets': e.nJets_OR, \
                             'nBJets': e.nJets_OR_MV2c10_70, \
                             'Mll01': e.Mll01/1000., \
                 }") \
    .AsAwkwardArray() \
    .value()

In [ ]:
import matplotlib.pyplot as plt

# Make some cuts: trilepton events, and perhaps leptons 0,1 or 0,2 are opposite-sign, same flavor
# (Ntuple is such that if total charge is +- 1, leptons 1 and 2 are same sign)
is3L = data['is3L']
OSSF01 = is3L & (data['lep_id_0'] == -data['lep_id_1'])
OSSF02 = is3L & (data['lep_id_0'] == -data['lep_id_2'])

# make matplotlib histograms with weights
# there are of course many other plotting packages one can use
plt.subplots(1,2, figsize=(12,5))
plt.subplot(1,2,1)
plt.hist(m(data[OSSF01], 0, 1), range=(0,200), bins=50, weights=data[OSSF01]['mcWeight'])
plt.xlabel('m(l0,l1) [GeV]')
plt.ylabel('Events/4 GeV')
plt.subplot(1,2,2)
plt.hist(m(data[OSSF02], 0, 2), range=(0,200), bins=50, weights=data[OSSF02]['mcWeight'])
plt.xlabel('m(l0,l2) [GeV]')
plt.ylabel('Events/4 GeV')
plt.show()

In [ ]:
# Fill a ROOT histogram, should we prefer that for visualization
import ROOT
h1 = ROOT.TH1F('h1', 'example', 50, 0, 200)
for val, weight in zip(m(data[OSSF01], 0, 1), data[OSSF01]['mcWeight']):
    h1.Fill(val, weight)
c1 = ROOT.TCanvas()
h1.Draw('HIST')
c1.Draw()

In [ ]:
# compare our computed M(l0, l1) with the precomputed one from the ntuple itself
plt.hist(m(data[OSSF01], 0, 1), range=(0,200), bins=50, weights=data[OSSF01]['mcWeight'], label='Locally computed')
plt.hist(data[OSSF01]['Mll01'], range=(0,200), bins=50, weights=data[OSSF01]['mcWeight'], histtype='step', linewidth=2, label='From ntuple')
plt.xlabel('m(l0,l1) [GeV]')
plt.ylabel('Events/4 GeV')
plt.legend()
plt.show()

In [ ]:
# Make a 2D matplotlib histogram
import awkward as ak
plt.figure(figsize=(6,6))
plt.hist2d(ak.to_numpy(m(data[is3L], 0, 1)), ak.to_numpy(m(data[is3L], 0, 2)), range=((0,250),(0,250)), bins=(100,100))
plt.xlabel('m(l0,l1) [GeV]')
plt.ylabel('m(l0,l2) [GeV]')
plt.show()

In [ ]:
# Apply cuts in ServiceX selection with Where clause
# Can significantly reduce size of download from ServiceX!
dataset_uproot = "user.kchoi:user.kchoi.ttHML_80fb_ttZ"
ds = ServiceXSourceUpROOT(dataset_uproot, "nominal")
data_cut = ds.Select("lambda e: {'lep_pt_0': e.lep_Pt_0/1000., \
                             'lep_pt_1': e.lep_Pt_1/1000., \
                             'lep_pt_2': e.lep_Pt_2/1000., \
                             'lep_eta_0': e.lep_Eta_0, \
                             'lep_eta_1': e.lep_Eta_1, \
                             'lep_eta_2': e.lep_Eta_2, \
                             'lep_phi_0': e.lep_Phi_0, \
                             'lep_phi_1': e.lep_Phi_1, \
                             'lep_phi_2': e.lep_Phi_2, \
                             'lep_id_0': e.lep_ID_0, \
                             'lep_id_1': e.lep_ID_1, \
                             'lep_id_2': e.lep_ID_2, \
                             'is3L': e.is3L, \
                             'mcWeight': e.mcWeightOrg, \
                             'nJets': e.nJets_OR, \
                             'nBJets': e.nJets_OR_MV2c10_70, \
                 }") \
    .Where('lambda e: e.is3L and e.nJets >= 4') \
    .AsAwkwardArray() \
    .value()

In [ ]:
# show ServiceX cuts give the same results as doing the cuts on the inclusive dataset
# blue filled histogram = ServiceX does cuts; orange unfilled histogram = client code does cuts

plt.subplots(1,2, figsize=(12,5))
plt.subplot(1,2,1)

# selection to apply to inclusive dataset
is3L_nJets = is3L & (data['nJets'] >= 4)

plt.hist(data_cut['nJets'], range=(1.5,8.5), bins=7, weights=data_cut['mcWeight'], label='ServiceX cut')
plt.hist(data[is3L_nJets]['nJets'], range=(1.5,8.5), bins=7, weights=data[is3L_nJets]['mcWeight'], histtype='step', linewidth=4, label='Local cut')
plt.xlabel('# jets')
plt.legend()
plt.subplot(1,2,2)
plt.hist(data_cut['nBJets'], range=(-0.5,4.5), bins=5, weights=data_cut['mcWeight'], label='ServiceX cut')
plt.hist(data[is3L_nJets]['nBJets'], range=(-0.5,4.5), bins=5, weights=data[is3L_nJets]['mcWeight'], histtype='step', linewidth=4, label='Local cut')
plt.xlabel('# b-jets')
plt.legend()
plt.show()